In [1]:
import pandas as pd
import numpy as np
import pickle
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
train_emb1=torch.load('/content/drive/My Drive/Chalearn Data/UntunedKimCNN/first4k_data.pt')

In [3]:
train_emb2=torch.load('/content/drive/My Drive/Chalearn Data/UntunedKimCNN/second4k_data.pt')

In [4]:
train_emb=torch.cat((train_emb1,train_emb2),dim=0)

In [5]:
type(train_emb)

torch.Tensor

In [6]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Chalearn Data/train-annotation.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/sample_data')

In [7]:
from zipfile import ZipFile

zip_file = '/content/drive/My Drive/Chalearn Data/val-annotation-e.zip'
password = 'zeAzLQN7DnSIexQukc9W'

with ZipFile(zip_file) as zf:
  zf.extractall("/content/sample_data",pwd=bytes(password,'utf-8'))

In [8]:
from zipfile import ZipFile

zip_file = '/content/drive/My Drive/Chalearn Data/test-annotation-e.zip'
password = 'zeAzLQN7DnSIexQukc9W'

with ZipFile(zip_file) as zf:
  zf.extractall("/content/sample_data",pwd=bytes(password,'utf-8'))

In [9]:
df_train = pd.read_pickle('/content/sample_data/annotation_training.pkl')
df_val= pd.read_pickle('/content/sample_data/annotation_validation.pkl')

In [10]:
df_train = pd.DataFrame.from_dict(df_train)
df_val = pd.DataFrame.from_dict(df_val)

In [11]:
df_train=df_train.append(df_val,ignore_index=True)

In [12]:
df_test = pd.read_pickle('/content/sample_data/annotation_test.pkl')
df_test=pd.DataFrame.from_dict(df_test)
df_test=df_test.reset_index(drop=True)
df_test

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness
0,0.485981,0.645833,0.681319,0.669903,0.626168,0.822222
1,0.616822,0.593750,0.692308,0.514563,0.570093,0.655556
2,0.467290,0.625000,0.560440,0.524272,0.514019,0.522222
3,0.411215,0.458333,0.714286,0.660194,0.570093,0.400000
4,0.317757,0.437500,0.384615,0.524272,0.448598,0.411111
...,...,...,...,...,...,...
1995,0.289720,0.312500,0.208791,0.135922,0.177570,0.300000
1996,0.719626,0.781250,0.670330,0.572816,0.728972,0.722222
1997,0.355140,0.395833,0.472527,0.446602,0.411215,0.677778
1998,0.467290,0.645833,0.527473,0.669903,0.560748,0.622222


In [13]:
test_emb=torch.load('/content/drive/My Drive/Chalearn Data/test_embeddings.pt')

In [14]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [15]:
df_train

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness
0,0.523364,0.552083,0.626374,0.601942,0.504673,0.488889
1,0.345794,0.375000,0.472527,0.582524,0.457944,0.366667
2,0.252336,0.291667,0.406593,0.485437,0.373832,0.511111
3,0.457944,0.489583,0.505495,0.398058,0.457944,0.377778
4,0.607477,0.489583,0.406593,0.621359,0.570093,0.622222
...,...,...,...,...,...,...
7995,0.570093,0.614583,0.494505,0.689320,0.626168,0.577778
7996,0.542056,0.541667,0.549451,0.669903,0.579439,0.666667
7997,0.551402,0.593750,0.560440,0.572816,0.504673,0.644444
7998,0.514019,0.552083,0.461538,0.572816,0.551402,0.733333


In [16]:
y_train=df_train.loc[:,['agreeableness']]
y_test=df_test.loc[:,['agreeableness']]
y_train=torch.tensor(np.array(y_train)).type(torch.float32)
y_test=torch.tensor(np.array(y_test)).type(torch.float32)
y_train

tensor([[0.6264],
        [0.4725],
        [0.4066],
        ...,
        [0.5604],
        [0.4615],
        [0.7253]])

In [17]:
train = torch.utils.data.TensorDataset(train_emb,y_train)
test = torch.utils.data.TensorDataset(test_emb,y_test)

In [18]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train, batch_size = 64, shuffle = True)
test_loader = DataLoader(test, batch_size = 64, shuffle = True)

In [41]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    #self.conv0=nn.Conv2d(in_channels=1,out_channels=100,kernel_size=(2,768))
    self.conv1=nn.Conv2d(in_channels=1,out_channels=100,kernel_size=(3,768))
    self.conv2=nn.Conv2d(in_channels=1,out_channels=100,kernel_size=(4,768))
    self.conv3=nn.Conv2d(in_channels=1,out_channels=100,kernel_size=(5,768))
    self.fc1=nn.Linear(300,12)
    self.fc21=nn.Linear(36,1)
    self.fc22=nn.Linear(36,1)
    self.fc23=nn.Linear(36,1)
    self.fc24=nn.Linear(36,1)
    self.fc25=nn.Linear(36,1)
    self.dropout=nn.Dropout(p=0.2)

  def forward(self,x):
      x = x.unsqueeze(1)
      #x0=F.relu(self.conv0(x)).squeeze(3)
      x1=F.relu(self.conv1(x)).squeeze(3)
      x2=F.relu(self.conv2(x)).squeeze(3)
      x3=F.relu(self.conv3(x)).squeeze(3)
      #x0=F.max_pool1d(x0,x0.size(2))
      x1=F.max_pool1d(x1,x1.size(2))
      x2=F.max_pool1d(x2,x2.size(2))
      x3=F.max_pool1d(x3,x3.size(2))
      x=torch.cat((x1.squeeze(dim=2),x2.squeeze(dim=2),x3.squeeze(dim=2)),dim=1)
      x=self.fc1(self.dropout(x))
      x=self.fc2(x)
      return x

In [42]:
model=Network()
model.cuda()

Network(
  (conv1): Conv2d(1, 100, kernel_size=(3, 768), stride=(1, 1))
  (conv2): Conv2d(1, 100, kernel_size=(4, 768), stride=(1, 1))
  (conv3): Conv2d(1, 100, kernel_size=(5, 768), stride=(1, 1))
  (fc1): Linear(in_features=300, out_features=12, bias=True)
  (fc2): Linear(in_features=12, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [43]:
lr=0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion1=nn.MSELoss()
criterion2=nn.L1Loss()
epochs=10
for i in range(epochs):
  model.train(True)
  running_loss=0
  for emb,label in train_loader:
    emb1=emb.to(device)
    label1=label.to(device)
    optimizer.zero_grad()
    output=model.forward(emb1)
    loss1=criterion1(output,label1)
    loss2=criterion2(output,label1)
    loss=(loss1+loss2)/2
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  else:
    print(f"Training loss: {running_loss/len(train_loader)}")

Training loss: 0.1297615977227688
Training loss: 0.06535268926620483
Training loss: 0.05950950655341149
Training loss: 0.05733256584405899
Training loss: 0.05359517022967339
Training loss: 0.05045074564218521
Training loss: 0.04739702519774437
Training loss: 0.04405378678441048
Training loss: 0.04049524746835232
Training loss: 0.039303059667348865


In [44]:
#print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions1=[]
true1=[]

# Predict 
for emb,label in test_loader:


  # Add batch to GPU
  #batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  #b_input_ids, b_input_mask, b_labels = batch
  emb1=emb.to(device)
  label1=label.to(device)

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model.forward(emb1)

  logits = outputs
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  predictions1.append(logits)
  label_ids1 = label1.to('cpu').numpy()
  # Store predictions and true labels
  #predictions.append(logits)
  true1.append(label_ids1)

print('    DONE.')


    DONE.


In [45]:
predictions11=[]
for i in range(len(predictions1)):
  for j in range(len(predictions1[i])):
    predictions11.append(predictions1[i][j])


In [46]:
true_labels1=[]
for i in range(len(true1)):
  for j in range(len(true1[i])):
    true_labels1.append(true1[i][j])

In [47]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(true_labels1, predictions11))  

0.1189254


In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(true_labels1, predictions11))  